In [ ]:
# Генерируем модульный RAG-проект с ручной реализацией, трассировкой и жёсткой привязкой к документам
# Установка библиотек
!pip install -U langchain langchain-google-genai google-generativeai chromadb sentence-transformers langchain-community beautifulsoup4 html2text
!pip install -U langchain-community

from langsmith import Client

import os
#import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
#splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

from langsmith import traceable
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
#
#
# 2. Загрузка и разбиение документа
urls = [
    "https://www.euronews.com/news/europe",
    "https://www.dw.com/en/top-stories/s-9097",
    "https://www.reuters.com/news/archive/worldNews",
    "https://apnews.com/hub/europe",
    "https://www.bbc.com/news/world/europe"
]
loader = WebBaseLoader(urls)
                                                                        #loader = WebBaseLoader("https://www.politico.eu/tag/eu-policy/")
                                                                        #("https://www.reuters.com/news/archive/worldNews")
                                                                        #--("https://en.wikipedia.org/wiki/History_of_Europe")
                                                                        #--("https://european-union.europa.eu/news-and-events_en")
docs = loader.load()
#os.environ["LANGCHAIN_API_KEY"] =
# from langsmith import Client
# client = Client()
# projects = client.list_projects()
# print("Доступные проекты LangSmith:")
# for p in projects:
#      print("-", p.name)

from langchain_core.documents import Document
docs.append(
    Document(
        page_content="Sturm Eowyn ereignete sich in Österreich im Dezember 2024.",
        metadata={"source": "manually"})
)
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(docs)

print(f"Загружено {len(split_docs)} фрагментов")      ## должно быть >= 50
print(type(split_docs[0]))  # <class 'langchain_core.documents.base.Document'>
#
#
!pip install -q beautifulsoup4 html2text   # site
# 1. Schlüssel einrichten/Настройка ключей
#
from google.colab import files
uploaded = files.upload()

import os
import logging

from dotenv import load_dotenv

load_dotenv("env.txt")            #C:/Users/sash-/GenerativeAI-II-Project/assignment-spec/env.txt")

#Config.setup()
# # Einrichten des Loggers/Настройка логгера
logging.basicConfig(level=logging.INFO)  # Или DEBUG, если нужно больше деталей

# Затем задаём API ключи из окружения
class Config:
   @staticmethod
   def setup():
        langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
        if langchain_api_key:
            os.environ["LANGCHAIN_API_KEY"] = langchain_api_key

        langsmith_endpoint = os.getenv("LANGSMITH_ENDPOINT")
        if langsmith_endpoint:
            os.environ["LANGCHAIN_ENDPOINT"] = langsmith_endpoint

        langsmith_project = os.getenv("LANGSMITH_PROJECT")
        if langsmith_project:
            os.environ["LANGCHAIN_PROJECT"] = langsmith_project

#        if os.getenv("LANGSMITH_TRACING", "false").lower() == "true":
#            os.environ["LANGCHAIN_TRACING_V2"] = "true"

Config.setup()
assert os.getenv("LANGCHAIN_API_KEY") is not None
import google.generativeai as genai
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
#
from dotenv import load_dotenv
load_dotenv("env.txt")

import os
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
#os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
#
print("LANGCHAIN_API_KEY =", os.getenv("LANGCHAIN_API_KEY"))
#print("LANGSMITH_API_KEY =", os.getenv("LANGSMITH_API_KEY"))

# 3. Индексация
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectordb = Chroma.from_documents(documents=split_docs, embedding=embedding, persist_directory="chroma_storage")
vectordb.persist()              # DIESE Zeile speichert die Datenbank auf der Chroma 2-Festplatte. Speichert alle Vektordarstellungen (Einbettungen) im Ordner ./chroma_storage./ЭТА строка сохраняет базу на диск Chroma 2  Сохраняет в неё все векторные представления (embedding)   в папке ./chroma_storage
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

# 4. Модель Gemini
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.4)

# 5. Вспомогательные функции
chat_history = []

def build_prompt(question, docs, system_prompt):
    context = "\\n\\n".join([d.page_content for d in docs])
    full_prompt = f"{system_prompt}\\n\\nКонтекст:\\n{context}\\n\\nВопрос: {question}\\nОтвет:"
    return full_prompt

from langsmith import traceable           #Трассировка и наблюдаемость через LangSmith или LangFuse

@traceable(name="ask_rag")
def ask_rag(question, system_prompt):
    logging.info(f"[RAG] Вопрос: {question}")

    docs = retriever.get_relevant_documents(question)
    logging.info(f"[RAG] Abgerufene Dokumente: {len(docs)}")

    if not docs:
        logging.warning("[RAG] Документы не найдены. Возвращаем стандартный ответ.")
        return "Keine Dokumente gefunden. Zurückgeben einer Standardantwort"

    prompt = build_prompt(question, docs, system_prompt)
    logging.debug(f"[RAG] Gebildet prompt:\n{prompt[:1000]}...")  # Ограничение на длину
    response = llm.invoke(prompt)

    logging.info(f"[RAG] Antwort: {response.content[:200]}...")  # Показываем начало ответа
    chat_history.append({"user": question, "answer": response})

    return response

SYSTEM_PROMPT_A = "Antworten Sie streng kontextbezogen. Fehlen Informationen, sagen Sie: ‚Ich weiß es nicht. Das stand nicht im Dokument"
SYSTEM_PROMPT_B = "Nutzen Sie den Kontext und ziehen Sie logische Schlussfolgerungen, aber fügen Sie keine Fakten von außen hinzu."
ask_rag("Sturm Eowyn, verheerender Wintersturm, in welchem Land ereignete er sich?", SYSTEM_PROMPT_A)
# 6. Примеры и сравнение
questions = [
    "Welche Maßnahmen wurden im Oktober 2024 in Deutschland ergriffen?",
    "Was geschah nach der US-Präsidentschaftswahl im November 2024?",
    "Welche Klimainitiativen wurden im Dezember 2024 gestartet?",
    "Was hat sich im September 2024 in der EU-Gesetzgebung geändert?",
    "Sturm Eowyn, verheerender Wintersturm, in welchem Land ereignete er sich?"
]

#SYSTEM_PROMPT_A = "Ответь строго на основе приведённого контекста. Если информации нет — скажи: 'Я не знаю. В документе этого не было.'"
#SYSTEM_PROMPT_B = "Используй контекст и делай логические выводы, но не добавляй факты извне."

print("\\n--- Modus A ---")
for q in questions:
    print(f"\\nFrage: {q}")
    print("Ответ:", ask_rag(q, SYSTEM_PROMPT_A))

print("\\n--- Modus B ---")
for q in questions:
    print(f"\\nFrage: {q}")
    print("Antwort:", ask_rag(q, SYSTEM_PROMPT_B))

# 7.Ohne Extraktion (Check)
print("\\n---Ohne Kontext (deaktivierter Retriever) ---")
def ask_without_context(question):
    prompt = f"Frage: {question}\\nAntwort:"
    return llm.invoke(prompt)

for q in questions:
    print(f"\\nFrage: {q}")
    print("Antwort:", ask_without_context(q))


  Using cached langchain_google_genai-2.1.4-py3-none-any.whl.metadata (5.2 kB)
  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.4-py3-none-any.whl.metadata (4.2 kB)
  Using cached google_generativeai-0.8.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.2-py3-none-any.whl.metadata (3.9 kB)
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
  Using cached google_generativeai-0.8.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.0-py3-none-any.whl.metadata (3.9 kB)
INFO: This is taking longer than usual. You might need to pro

Saving env.txt to env (9).txt
LANGCHAIN_API_KEY = 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


\n--- Modus A ---
\nFrage: Welche Maßnahmen wurden im Oktober 2024 in Deutschland ergriffen?
Ответ: content='Ich weiß es nicht. Das stand nicht im Dokument' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--387504ae-5936-49ac-b2de-c33ff2a37be2-0' usage_metadata={'input_tokens': 165, 'output_tokens': 11, 'total_tokens': 176, 'input_token_details': {'cache_read': 0}}
\nFrage: Was geschah nach der US-Präsidentschaftswahl im November 2024?
Ответ: content='Ich weiß es nicht. Das stand nicht im Dokument' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--70d3c9c2-492c-4a9a-99c8-64d8e2c2a18d-0' usage_metadata={'input_tokens': 170, 'output_tokens': 11, 'total_tokens': 181, 'input_token_details': {'cache_read': 0}}
\nFrage: Welche Klimainitiativen wurden im Dezember 2024 gestartet?
О

Antwort: content='Basierend auf dem Kontext lässt sich keine logische Schlussfolgerung über Maßnahmen in Deutschland im Oktober 2024 ziehen. Der Kontext liefert ausschließlich Informationen über einen Sturm in Österreich im Dezember 2024.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--93df94cd-8369-4e15-b234-9320a6f9ecdb-0' usage_metadata={'input_tokens': 162, 'output_tokens': 45, 'total_tokens': 207, 'input_token_details': {'cache_read': 0}}
\nFrage: Was geschah nach der US-Präsidentschaftswahl im November 2024?
Antwort: content='Antwort: In Österreich ereignete sich im Dezember 2024 der Sturm Eowyn.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--3cb161b4-5902-4a06-b4e0-8998bae2e442-0' usage_metadata={'input_tokens': 167, 'output_tokens': 22, 'total_tokens': 189, '

Antwort: content='Basierend auf dem Kontext kann ich keine Informationen über Klimainitiativen ableiten, die im Dezember 2024 gestartet wurden. Der Kontext liefert ausschließlich die Information, dass im Dezember 2024 der Sturm Eowyn in Österreich stattfand.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--21a30eea-12e2-4c74-9e6c-50add9089b48-0' usage_metadata={'input_tokens': 162, 'output_tokens': 55, 'total_tokens': 217, 'input_token_details': {'cache_read': 0}}
\nFrage: Was hat sich im September 2024 in der EU-Gesetzgebung geändert?


Antwort: content='Aufgrund des gegebenen Kontexts kann ich keine Aussage darüber treffen, was sich im September 2024 in der EU-Gesetzgebung geändert hat. Der Kontext bezieht sich ausschließlich auf den Sturm Eowyn in Österreich im Dezember 2024 und enthält keine Informationen über EU-Gesetzgebung.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--6cddf675-bb61-4eac-b18e-75ddd742f994-0' usage_metadata={'input_tokens': 166, 'output_tokens': 65, 'total_tokens': 231, 'input_token_details': {'cache_read': 0}}
\nFrage: Sturm Eowyn, verheerender Wintersturm, in welchem Land ereignete er sich?
Antwort: content='Österreich.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--e5bfe535-9184-4c77-af7e-90550d29e783-0' usage_metadata={'input_tokens': 167, 'output_tokens': 3, 'total_token

Antwort: content='Um Ihnen eine genaue Antwort zu geben, benötige ich mehr Informationen darüber, welche Art von Maßnahmen Sie interessieren.  Im Oktober 2024 können viele verschiedene Dinge passiert sein.  Um Ihnen zu helfen, geben Sie mir bitte mehr Kontext.  Zum Beispiel:\n\n*   **Welches Themengebiet interessiert Sie?** (z.B. Wirtschaft, Politik, Gesundheit, Umwelt, Technologie, Kultur, Sport, etc.)\n*   **Welche Art von Maßnahmen suchen Sie?** (z.B. Gesetze, Verordnungen, politische Entscheidungen, Proteste, Veranstaltungen, etc.)\n*   **Gibt es bestimmte Regionen oder Städte, die Sie interessieren?**\n\nJe genauer Ihre Frage, desto besser kann ich Ihnen helfen.\n\nOhne weitere Informationen kann ich Ihnen nur sehr allgemeine Dinge nennen, die wahrscheinlich im Oktober 2024 in Deutschland passiert sind:\n\n*   **Alltägliche Regierungsarbeit:** Die Bundesregierung hat wahrscheinlich weiterhin Gesetze vorbereitet, diskutiert und verabschiedet.\n*   **Veranstaltungen:** Es gab wahrsc

Antwort: content='Die Antwort auf diese Frage kann ich dir erst nach der US-Präsidentschaftswahl im November 2024 geben.  Nach der Wahl werden die Ergebnisse bekannt sein, und man wird wissen, wer zum Präsidenten der Vereinigten Staaten gewählt wurde.  Danach beginnt eine Übergangsphase, in der der gewählte Präsident sein Kabinett zusammenstellt und sich auf die Amtsübernahme vorbereitet.  Die tatsächliche Amtsübernahme findet dann im Januar 2025 statt.\n\nUm eine vollständige Antwort zu geben, müsste man also die folgenden Punkte berücksichtigen:\n\n*   **Wer wurde gewählt?**\n*   **Wie war die Reaktion der Öffentlichkeit und der politischen Parteien auf das Ergebnis?**\n*   **Welche wichtigen Personalentscheidungen wurden getroffen?**\n*   **Welche politischen Prioritäten hat der gewählte Präsident für seine Amtszeit angekündigt?**\n\nBitte frag mich nach der Wahl im November 2024 noch einmal!' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_rat

Antwort: content='Um Ihnen eine genaue Antwort auf die Frage nach Klimainitiativen zu geben, die im Dezember 2024 gestartet wurden, benötige ich weitere Informationen.  Ich habe keinen Echtzeit-Zugriff auf Informationen und kann daher nicht wissen, welche Initiativen im Dezember 2024 gestartet wurden.\n\nUm Ihnen dennoch weiterzuhelfen, kann ich Ihnen folgendes anbieten:\n\n*   **Allgemeine Informationen über Klimainitiativen:** Ich kann Ihnen generelle Informationen über verschiedene Arten von Klimainitiativen geben, z.B. solche, die sich auf erneuerbare Energien, Energieeffizienz, Reduzierung von Emissionen, Anpassung an den Klimawandel oder Klimafinanzierung konzentrieren.\n*   **Beispiele für Klimainitiativen:** Ich kann Ihnen Beispiele für bereits existierende Klimainitiativen nennen, die von Regierungen, internationalen Organisationen, Unternehmen oder NGOs durchgeführt werden.\n*   **Wo Sie Informationen finden können:** Ich kann Ihnen Quellen nennen, wo Sie nach aktuellen Infor

Antwort: content='Im September 2024 gab es einige wichtige Änderungen in der EU-Gesetzgebung, die je nach Bereich variieren können.  Hier sind einige Beispiele, die je nach deinem Interessensgebiet relevant sein könnten:\n\n*   **Digitales:** Das **Gesetz über digitale Märkte (DMA)** und das **Gesetz über digitale Dienste (DSA)** sind seit einiger Zeit in Kraft, aber im September 2024 könnten weitere Durchsetzungsmaßnahmen oder Klarstellungen erwartet werden. Diese Gesetze zielen darauf ab, die Marktmacht von großen Online-Plattformen zu regulieren und die Sicherheit der Nutzer im Internet zu verbessern.\n\n*   **Nachhaltigkeit:** Es gab und gibt weiterhin viele Änderungen im Bereich der Nachhaltigkeit. Möglicherweise gab es im September 2024 neue Berichtsstandards für Unternehmen, die im Einklang mit der **Corporate Sustainability Reporting Directive (CSRD)** stehen. Auch Änderungen in Bezug auf die **EU-Taxonomie** (Klassifizierung nachhaltiger Wirtschaftsaktivitäten) sind möglich.\n